# Creatives Dashboard

## 1. Run the following cell: 
### - Enter Redshift Username and Password 
### - Select Dates

In [1]:
import psycopg2
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
import pickle
from pandas.api.types import is_numeric_dtype
import ipywidgets as widgets
from IPython.display import display, clear_output, Javascript
# pip install --user psycopg2
#pip install ipython-sql
import sqlalchemy

user = input("Enter RedshiftDB username: ")
password = input("Enter RedshiftDB password: ")
dbname = 'gamecircus'
host = 'gc-dwh.cxykxux8ksim.us-east-1.redshift.amazonaws.com'
port = '5439'
conn = psycopg2.connect(dbname=dbname, user=user, password=password, host=host, port=port)


#****************************************************************DATA LOADING*********************************************************************

start_date_picker = widgets.DatePicker(description='Select Install Cohort Start Date', disabled=False)
end_date_picker = widgets.DatePicker(description='Select Install Cohort End Date', disabled=False)
display(start_date_picker, end_date_picker)

Enter RedshiftDB username: dkhan
Enter RedshiftDB password: <PJR&w2(8gcE9j


DatePicker(value=None, description='Select Install Cohort Start Date', step=1)

DatePicker(value=None, description='Select Install Cohort End Date', step=1)

## Run the following cell and select from the menu options
*Note: this might take about a minute depending on the length of timeframe selected*

In [3]:
pd.set_option('display.max_columns', 100)
start_date = start_date_picker.value.strftime('%Y-%m-%d')
end_date = end_date_picker.value.strftime('%Y-%m-%d')

creatives_script = f'''with xpend as (
select app_id, media_source, campaign, ad, adset, sum(impressions) as impressions, sum(clicks) as clicks, sum(installs) as installs, sum(cost) as cost
from xpend_appsflyer_etl xae where date between '{start_date}' and '{end_date}' and app_id in ('com.leftover.CoinDozer', 'id372836496', 'com.gamecircus.tycoon', 'com.gamecircus.CoinDozerJackpot')
group by app_id, media_source, campaign, ad, adset
),

af_events as (
select customer_user_id, campaign, af_ad, af_adset from appsflyer_events_new aen 
where app_id in ('com.leftover.CoinDozer', 'id372836496', 'com.gamecircus.tycoon', 'com.gamecircus.CoinDozerJackpot') 
and install_time::date between '{start_date}' and '{end_date}' and event_name in ('install')
),

dx_revenue as (
select a.campaign, af_ad, af_adset, sum(D0) as D0, sum(D3) as D3, sum(D7) as D7, sum(D14) as D14, sum(DNow) as DNow from 
(
select primary_id, campaign, 
case when report_date = install_date then sum(revenue) end as D0,
case when report_date - install_date <=3 then sum(revenue) end as D3,
case when report_date - install_date <=7 then sum(revenue) end as D7,
case when report_date - install_date <=30 then sum(revenue) end as D14,
case when report_date - install_date <=30 then sum(revenue) end as D30,
case when (report_date - install_date) <= (current_date - install_date) then sum(revenue) end as DNow
from revenue_user_summary rus 
where app_id in ('com.leftover.CoinDozer', 'id372836496', 'com.gamecircus.tycoon', 'com.gamecircus.CoinDozerJackpot') 
and install_date between '{start_date}' and '{end_date}'
group by primary_id, campaign, report_date, install_date
) a left outer join af_events on a.campaign = af_events.campaign and a.primary_id = af_events.customer_user_id
group by a.campaign, af_ad, af_adset
)

select app_id, media_source, xpend.campaign, ad, adset, impressions, clicks, installs, cost, 
D0 as D0_revenue, D3 as D3_revenue , D7 as D7_revenue, D14 as D14_revenue, DNow as total_revenue
from xpend --left outer join rus_cte on xpend.campaign = rus_cte.campaign and (xpend.ad = rus_cte.af_ad and xpend.adset = rus_cte.af_adset)
LEFT JOIN dx_revenue ON xpend.campaign = dx_revenue.campaign AND
         (
            (dx_revenue.af_ad IS NOT NULL AND dx_revenue.af_adset IS NOT NULL AND xpend.ad = dx_revenue.af_ad AND xpend.adset = dx_revenue.af_adset)
            OR 
            (dx_revenue.af_ad IS NULL AND dx_revenue.af_adset IS not NULL AND xpend.adset = dx_revenue.af_adset)
            or
            (dx_revenue.af_ad IS NOT NULL AND dx_revenue.af_adset IS NULL AND xpend.ad = dx_revenue.af_ad)
        )'''


df_creatives = pd.read_sql_query(creatives_script, conn)
conn.close()

df = df_creatives.copy()

df['d0_arpu'] = np.where((df['installs'] != 0) & (~df['d0_revenue'].isnull()), df['d0_revenue'] / df['installs'], 0)
df['d0_arpu'] = df['d0_arpu'].round(2)

df['d3_arpu'] = np.where((df['installs'] != 0) & (~df['d3_revenue'].isnull()), df['d3_revenue'] / df['installs'], 0)
df['d3_arpu'] = df['d3_arpu'].round(2)

df['d7_arpu'] = np.where((df['installs'] != 0) & (~df['d7_revenue'].isnull()), df['d7_revenue'] / df['installs'], 0)
df['d7_arpu'] = df['d7_arpu'].round(2)

df['d14_arpu'] = np.where((df['installs'] != 0) & (~df['d14_revenue'].isnull()), df['d14_revenue'] / df['installs'], 0)
df['d14_arpu'] = df['d14_arpu'].round(2)

df['eCPI'] = np.where((df['installs'] != 0) & (~df['cost'].isnull()), df['cost'] / df['installs'], 0)
df['eCPI'] = df['eCPI'].round(2)

df['d0_roas'] = np.where((df['cost'] != 0) & (~df['d0_revenue'].isnull()), df['d0_revenue'] / df['cost'], 0)
df['d0_roas'] = (df['d0_roas'] * 100).round(2).astype(str) + '%'

df['d3_roas'] = np.where((df['cost'] != 0) & (~df['d3_revenue'].isnull()), df['d3_revenue'] / df['cost'], 0)
df['d3_roas'] = (df['d3_roas'] * 100).round(2).astype(str) + '%'

df['d7_roas'] = np.where((df['cost'] != 0) & (~df['d7_revenue'].isnull()), df['d7_revenue'] / df['cost'], 0)
df['d7_roas'] = (df['d7_roas'] * 100).round(2).astype(str) + '%'

df['d14_roas'] = np.where((df['cost'] != 0) & (~df['d14_revenue'].isnull()), df['d14_revenue'] / df['cost'], 0)
df['d14_roas'] = (df['d14_roas'] * 100).round(2).astype(str) + '%'


df['d0_revenue'] = df['d0_revenue'].round(2)
df['d3_revenue'] = df['d3_revenue'].round(2)
df['d7_revenue'] = df['d7_revenue'].round(2)
df['d14_revenue'] = df['d14_revenue'].round(2)
df['total_revenue'] = df['total_revenue'].round(2)
df['cost'] = df['cost'].round(2)

df['CTR'] = np.where((df['impressions'] != 0) & (~df['clicks'].isnull()), df['clicks'] / df['impressions'], 0)
df['CTR'] = (df['CTR'] * 100).round(2).astype(str) + '%'

df['CVR'] = np.where((df['clicks'] != 0) & (~df['impressions'].isnull()), df['installs'] / df['clicks'], 0)
df['CVR'] = (df['CVR'] * 100).round(2).astype(str) + '%'

df['IR'] = np.where((df['impressions'] != 0) & (~df['installs'].isnull()), df['installs'] / df['impressions'], 0)
df['IR'] = (df['IR'] * 100).round(2).astype(str) + '%'

df['IPM'] = np.where((df['impressions'] != 0) & (~df['installs'].isnull()), df['installs'] / (df['impressions'] / 1000), 0)
df['IPM'] = df['IPM'].round(2)

df['CPM'] = np.where((df['impressions'] != 0) & (~df['cost'].isnull()), (df['cost'] / (df['impressions']) * 1000), 0)
df['CPM'] = df['CPM'].round(2)

# Result by Creatives

def search_by_ad(ad):
    return df[df['ad']==ad].sort_values('impressions', ascending = False)

def search_by_adset(adset):
    return df[df['adset']==adset].sort_values('impressions', ascending = False)

def search_by_campaign(campaign):
    return df[df['campaign']==campaign].sort_values(['impressions'], ascending = False)


#***************************************************************************************************************************

import ipywidgets as widgets
from IPython.display import display

# Dropdown for Adset
dropdown_ad = widgets.Combobox(
    options=df['ad'].unique().tolist(),
    placeholder='Type to search',
    description='Select an option:',
    ensure_option=True,  # Ensure the typed value is one of the options
    disabled=False
)

# Function to handle dropdown value changes
def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print(f'Selected: {change["new"]}')

dropdown_ad.observe(on_change)

#######################################################################################################

# Dropdown for Adset

dropdown_adset = widgets.Combobox(
    options=df['adset'].unique().tolist(),
    placeholder='Type to search',
    description='Select an option:',
    ensure_option=True,  # Ensure the typed value is one of the options
    disabled=False
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print(f'Selected: {change["new"]}')

dropdown_adset.observe(on_change)
#######################################################################################################

dropdown_campaign = widgets.Combobox(
    options=df['campaign'].unique().tolist(),
    placeholder='Type to search',
    description='Select an option:',
    ensure_option=True,  # Ensure the typed value is one of the options
    disabled=False
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print(f'Selected: {change["new"]}')

dropdown_campaign.observe(on_change)


#***************************************************************************************************************************

# Display Buttons
run_button = widgets.Button(description='Get Report')
by_campaign_button = widgets.Button(description='By Campaign')
by_creative_button = widgets.Button(description='By Creatives')
ad_button = widgets.Button(description='By Ad')
adset_button = widgets.Button(description='By Adset')
main_menu_button = widgets.Button(description='Main Menu')

# Flow for Button Clicks

# Flow for Creative Button
def function_by_creative_button(button_click):
    clear_output()
    display(ad_button, adset_button, main_menu_button)

def function_by_ad(button_click):
    clear_output()
    display(dropdown_ad, run_button, main_menu_button)
    
    def run_function(button_click):
        clear_output()
        display(search_by_ad(dropdown_ad.value), main_menu_button)
    run_button.on_click(run_function)
        

def function_by_adset(button_click):
    clear_output()
    display(dropdown_adset, run_button, main_menu_button)
    
    def run_function(button_click):
        clear_output()
        display(search_by_adset(dropdown_adset.value), main_menu_button)
    run_button.on_click(run_function)

    
# Flow for Campaign Button
def function_by_campaign(button_click):
    clear_output()
    display(dropdown_campaign, run_button, main_menu_button)
    
    def run_function(button_click):
        clear_output()
        display(search_by_campaign(dropdown_campaign.value), main_menu_button)
    run_button.on_click(run_function)


def function_main_menu(button_click):
    clear_output()
    display(by_campaign_button,by_creative_button)
    
#     def run_function(button_click):
#         clear_output()
#         display(search_by_campaign(dropdown_campaign.value))
#     run_button.on_click(run_function)
    
ad_button.on_click(function_by_ad)
adset_button.on_click(function_by_adset)
by_campaign_button.on_click(function_by_campaign)

# Main Menu Display
display(by_campaign_button,by_creative_button)

by_campaign_button.on_click(function_by_campaign)
by_creative_button.on_click(function_by_creative_button)
main_menu_button.on_click(function_main_menu)

Button(description='By Campaign', style=ButtonStyle())

Button(description='By Creatives', style=ButtonStyle())